In [1]:
import os
os.chdir("..")

In [2]:
import world_bank_data as wb
import pandas as pd
import polars as pl
import numpy as np
import pandas as pd
from src.data.data_pull import DataClean

dc = DataClean()

# TODO
- colapse by 5 average
- keep the balance data
- 

In [3]:
dc.pull_wb()

year,country,gdp,gdp_growth,spending,gini,expenses
i32,str,f32,f32,f32,f32,f32
1960,"""Africa Eastern and Southern""",2.4211e10,null,5.966216,null,null
1960,"""Africa Western and Central""",1.1905e10,null,null,null,null
1960,"""Arab World""",null,null,null,null,null
1960,"""Caribbean small states""",6.41146368e8,null,null,null,null
1960,"""Central Europe and the Baltics""",null,null,null,null,null
…,…,…,…,…,…,…
2024,"""Virgin Islands (U.S.)""",null,null,null,null,null
2024,"""West Bank and Gaza""",null,null,null,null,null
2024,"""Yemen, Rep.""",null,null,null,null,null


In [4]:
df = pl.DataFrame(wb.get_countries())
df

iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
str,str,str,str,str,str,str,f64,f64
"""AW""","""Aruba""","""Latin America & Caribbean ""","""""","""High income""","""Not classified""","""Oranjestad""",-70.0167,12.5167
"""ZH""","""Africa Eastern and Southern""","""Aggregates""","""""","""Aggregates""","""Aggregates""","""""",null,null
"""AF""","""Afghanistan""","""South Asia""","""South Asia""","""Low income""","""IDA""","""Kabul""",69.1761,34.5228
"""A9""","""Africa""","""Aggregates""","""""","""Aggregates""","""Aggregates""","""""",null,null
"""ZI""","""Africa Western and Central""","""Aggregates""","""""","""Aggregates""","""Aggregates""","""""",null,null
…,…,…,…,…,…,…,…,…
"""A5""","""Sub-Saharan Africa excluding S…","""Aggregates""","""""","""Aggregates""","""Aggregates""","""""",null,null
"""YE""","""Yemen, Rep.""","""Middle East & North Africa""","""Middle East & North Africa (ex…","""Low income""","""IDA""","""Sana'a""",44.2075,15.352
"""ZA""","""South Africa""","""Sub-Saharan Africa ""","""Sub-Saharan Africa (excluding …","""Upper middle income""","""IBRD""","""Pretoria""",28.1871,-25.746


In [5]:
df = dc.wb_data(params=["GFDD.OI.01"],year=1980)
df

country,GFDD.OI.01,year
str,str,i32
"""Aruba""",null,1980
"""Afghanistan""",null,1980
"""Angola""",null,1980
"""Albania""",null,1980
"""Andorra""",null,1980
…,…,…
"""Kosovo""",null,1980
"""Yemen, Rep.""",null,1980
"""South Africa""",null,1980


In [6]:
columns = [
    'gdp_growth', 'spending', 'gini',
    'expenses'
]
data = df.to_pandas()

# Step 1: Sort the DataFrame
data = data[data["country"] != "World"]
data = data[data["country"] == "United States"]
data = data.sort_values(by=['year', 'country']).reset_index(drop=True)

# Step 2: Interpolate each column by zip group
# data[columns] = data.groupby('country')[columns].transform(
#     lambda group: group.interpolate(method='linear', limit_direction='both')
# )
data = data.dropna(subset=["gdp_growth"]).reset_index(drop=True)
data = data.dropna(subset=["expenses"]).reset_index(drop=True)
data

KeyError: ['gdp_growth']

In [ ]:
def plot_knots(knots, ax):
    for knot in knots:
        ax.axvline(knot, color="0.1", alpha=0.4)
    return ax

In [ ]:
num_knots = 5
knots = np.quantile(data["expenses"], np.linspace(0, 1, num_knots))
iknots = knots[1:-1]
priors = {
    "Intercept": bmb.Prior("Normal", mu=100, sigma=10),
    "common": bmb.Prior("Normal", mu=0, sigma=10), 
    "sigma": bmb.Prior("Exponential", lam=1)
}
model = bmb.Model("gdp_growth ~ year + bs(expenses, knots=iknots, intercept=True)", data, dropna=True, priors=priors)

In [ ]:
def plot_spline_basis(basis, expenses, figsize=(10, 6)):
    df = (
        pd.DataFrame(basis)
        .assign(expenses=expenses)
        .melt("expenses", var_name="basis_idx", value_name="gdp_growth")
    )

    _, ax = plt.subplots(figsize=figsize)

    for idx in df.basis_idx.unique():
        d = df[df.basis_idx == idx]
        ax.plot(d["expenses"], d["gdp_growth"])
    
    return ax

In [ ]:
B = model.components["mu"].design.common["bs(expenses, knots=iknots, intercept=True)"]
ax = plot_spline_basis(B, data["expenses"].values)
plot_knots(knots, ax);

In [ ]:
# The seed is to make results reproducible
idata = model.fit(idata_kwargs={"log_likelihood": True})

In [ ]:
az.plot_trace(idata);

In [ ]:
posterior_stacked = az.extract(idata)
wp = posterior_stacked["bs(expenses, knots=iknots, intercept=True)"].mean("sample").values

ax = plot_spline_basis(B * wp.T, data["expenses"].values)
ax.plot(data.expenses.values, np.dot(B, wp.T), color="black", lw=3)
plot_knots(knots, ax);